[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/quick-tour/interacting-with-the-index.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/quick-tour/interacting-with-the-index.ipynb)

# Interacting with a Pinecone index

Pinecone creates an index for your input vectors,
and it lets you query their nearest neighbors.
A Pinecone index supports the following operations:

* `upsert`: insert data formatted as `(id, vector)` tuples into the index, or replace existing `(id, vector)` tuples with new vector values. Optionally, you can attach metadata for each vector so you can use them in the query by specifying conditions. The upserted vector will look like `(id, vector, metadata)`.
* `delete`: delete vectors by id.
* `query`: query the index and retrieve the top-k nearest neighbors based on dot-product, cosine-similarity, Euclidean distance, and more. 
* `fetch`: fetch vectors stored in the index by id.
* `describe_index_stats`: get statistics about the index.

## Prerequisites

Install dependencies.

In [1]:
!pip install -qU pinecone-client pandas

Set up Pinecone.

In [2]:
import pinecone
import os

# Load Pinecone API key
api_key = os.getenv('PINECONE_API_KEY') or 'YOUR_API_KEY'
pinecone.init(
    api_key=api_key,
    environment="YOUR_ENV"  # find next to API key in console
)

## Creating the index

In [4]:
index_name = "pinecone-example"

# Delete index if exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

# Create index
pinecone.create_index(name=index_name, dimension=2, metric="euclidean", shards=1)

# Connect to the index
index = pinecone.Index(index_name=index_name)

### Insert vectors


In [5]:
# Generate some data
import pandas as pd

df = pd.DataFrame()
df["id"] = ["A", "B", "C", "D", "E"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.], [5., 5.]]
df

,id,vector
0,A,"[1.0, 1.0]"
1,B,"[2.0, 2.0]"
2,C,"[3.0, 3.0]"
3,D,"[4.0, 4.0]"
4,E,"[5.0, 5.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [6]:
# Upsert the vectors
AB_df = df[:2]
index.upsert(vectors=zip(AB_df.id, AB_df.vector))

{'upserted_count': 2}

### Fetch vectors

In [7]:
# Fetch vectors by ID
fetch_results = index.fetch(ids=["A", "B"])
fetch_results

{'namespace': '',
 'vectors': {'A': {'id': 'A', 'values': [0.99999994, 0.99999994]},
             'B': {'id': 'B', 'values': [1.99999988, 1.99999988]}}}

### Query top-k vectors

In [8]:
# Query top-k nearest neighbors
query_results = index.query(queries=[[1.1, 1.1], [2.2, 2.2]], top_k=2)
query_results

{'results': [{'matches': [{'id': 'B', 'score': 1.6200006, 'values': []},
                          {'id': 'A', 'score': 0.0200003274, 'values': []}],
              'namespace': ''},
             {'matches': [{'id': 'B', 'score': 0.0800013095, 'values': []},
                          {'id': 'A', 'score': 2.88000083, 'values': []}],
              'namespace': ''}]}

### Update vectors by ID

In [9]:
# Fetch current vectors by ID
fetch_result = index.fetch(ids=["A"])
fetch_result

{'namespace': '',
 'vectors': {'A': {'id': 'A', 'values': [0.99999994, 0.99999994]}}}

In [10]:
# Update vectors by ID
index.upsert(vectors=[("A",[0., 0.])])

{'upserted_count': 1}

In [11]:
# Fetch vector by the same ID again
fetch_result = index.fetch(ids=["A"])
fetch_result

{'namespace': '', 'vectors': {'A': {'id': 'A', 'values': [0.0, 0.0]}}}

### Delete vectors by ID

In [12]:
# Delete vectors by ID
index.delete(ids=["A"])

{}

In [13]:
# Deleted vectors are empty
fetch_results = index.fetch(ids=["A", "B"])
fetch_results

{'namespace': '',
 'vectors': {'B': {'id': 'B', 'values': [1.99999988, 1.99999988]}}}

### Get index statistics

In [14]:
# Index statistics
index.describe_index_stats()

{'dimension': 2, 'namespaces': {'': {'vector_count': 1}}}

### Delete the index

In [15]:
# delete the index
pinecone.delete_index(index_name)